**Note:** for running the example we need at least a `ml.m5.large` instance

# SETUP

## Check the version of python using

In [3]:
!which python3

/opt/conda/bin/python3


## Install required libraries

In [3]:
!pip install --upgrade pip

In [5]:
!pip install ipywidgets --quiet

In [6]:
!pip install boto3 --quiet
!pip install langchain --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.14.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a6 which is incompatible.


In [7]:
!pip install torch --quiet

In [8]:
!pip install sentence_transformers --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.4 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.


In [9]:
!pip install chromadb --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.


In [11]:
!pip install typing-extensions --upgrade

In [12]:
!pip install SQLAlchemy==2.0.1 --quiet
!pip install flask-sqlalchemy --quiet



In [13]:
!pip install unstructured --quiet

In [14]:
!pip install pdf2image
!pip install pdfminer
!pip install pdfminer.six

  Using cached pdf2image-1.16.3-py3-none-any.whl (11 kB)
  Using cached pdfminer-20191125-py3-none-any.whl
  Obtaining dependency information for pycryptodome from https://files.pythonhosted.org/packages/40/88/89623388754162ddb82c62814ccbfdbfcc8ed9bd6d5f7412d2479bdca3a7/pycryptodome-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pycryptodome-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached pycryptodome-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)


In [4]:
!pip install pyrouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191604 sha256=ec4ea150cdd73c3f0c94c3917d07461a57e8a690d82ffcd59bbdd34cdf77903a
  Stored in directory: /root/.cache/pip/wheels/9a/67/12/c5dd8ef8b4152bb8789eafd2a74a734e2dc7bb9eae02b768e7
Successfully built pyrouge


## Import libraries

In [15]:
import chromadb 

In [16]:
import sentence_transformers

In [17]:
import langchain

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import NLTKTextSplitter
import os

## Setup the llm 

In [47]:
import boto3
region = boto3.Session().region_name
endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-7b-3"

content_handler = ContentHandler()

llm=SagemakerEndpoint(
     endpoint_name=endpoint_name, 
     region_name=region, 
     model_kwargs={"max_new_tokens": 4092, "top_p": 0.9, "temperature": 0.01},
     endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
     content_handler=content_handler
 )


### ContentHandler for LLama-2-chat

In [ ]:
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs" : [[{"role" : "system",
        "content" : "Your are a kind assistant for the supreme court. All your generted  answer MUST be in grammatically correct spanish."},
        {"role" : "user", "content" : prompt}]],
        "parameters" : {**model_kwargs}})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generation"]["content"]

### Test the LLM

In [ ]:
print(llm("tell me a joke"))

## Setup SBERT for calculating embeddings

In [19]:
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [20]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

# INTERACTIVE EXAMPLE

## Load a PDF and calculate the embeddings

In [21]:
loader = UnstructuredPDFLoader("/root/JurisGPT/rawdata/laboral/10000003368.pdf")

In [22]:
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])

In [51]:
index.vectorstore.delete_collection()

## Ask LLM to summarize each section

In [39]:
query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Antecedentes' of a particular court ruling . Write your answer in spanish in no less than 150 words:"
summary_ant = (index.query(query,llm=llm))
print(summary_ant)


 En la sección "Antecedentes" del fallo de la Suprema Corte de Justicia de Mendoza en la causa "Manrique" (N° 13-04491180-3/1), se resume el contexto legal que se encuentra en la provincia de Mendoza. Se menciona que existen pronunciamientos de los integrantes de la Sala II en favor de la inconstitucionalidad del precepto en la misma causa, y que se reitera en otros pronunciamientos. Además, se destaca la necesidad de otorgar certeza en el modo de actuar y asegurar el tratamiento igualitario del justiciable que recurre ante el Poder Judicial en busca de un pronunciamiento.

Se analiza el marco normativo general y el texto legal objeto de análisis, destacando la adhesión de la Provincia de Mendoza a las disposiciones contenidas en el Título I de la Ley Nacional N° 27.348 (complementaria de la Ley sobre Riesgos del Trabajo N° 24.557), y la delegación de competencias a la jurisdicción administrativa nacional para dar cumplimiento a lo dispuesto en los artículos 1, 2 y 3 de la norma precit

In [16]:
query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la primera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
summary_1 = (index.query(query,llm=llm))
print(summary_1)


 La sección "Sobre la primera cuestión" del fallo del Tribunal de la Suprema Corte de Justicia de Mendoza se refiere a la admisión de un recurso extraordinario provincial interpuesto por Dante Ramón Farconesi contra la sentencia dictada en los autos n°159.313. Farconesi interpuso el recurso debido a que la sentencia inicial fue dictada en favor de Provincia ART S.A. en un caso de enfermedad accidental, originario de la Excma. Cámara Séptima del Trabajo de la Primera Circunscripción Judicial.

El Tribunal consideró que no hubo lesión alguna a la defensa de Farconesi en el juicio, ya que el informe pericial contable rola a fs. 233 muestra que la póliza entre el Gobierno de Mendoza y Provincia ART S.A. fue celebrada en fecha 01/09/2013. Además, la defensa formuló su defensa por intermedio del Dr. Miguel Grosso.

El Tribunal admitió el remedio intentado por Farconesi y ordenó el traslado a la contraria y la suspensión de los procedimientos en el grado. El Fiscal Adjunto Civil, Procuración 

In [17]:
query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la segunda cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
summary_2 = (index.query(query,llm=llm))
print(summary_2)

 En la sección "Sobre la segunda cuestión" del fallo del Tribunal de la Suprema Corte de Justicia de Mendoza, se analiza la admisión de la articulación interpuesta por Dante Ramón Farconesi en su recurso extraordinario provincial contra la sentencia dictada en el caso "Farconesi, Dante Ramón c/ Provincia ART SA p/ Enfermedad Accidente". La cuestión en cuestión es si la póliza entre el Gobierno de Mendoza y Provincia ART S.A. fue celebrada en fecha 01/09/2013, como afirma el informe pericial contable.

El Tribunal consideró que no hay lesión alguna a la defensa de Dante Ramón Farconesi en juicio, ya que el informe pericial no es contrario a la versión de los hechos presentada por la acción, sino más bien complementa y fortalece la misma. Además, se destacó que la admisión de la articulación es necesaria para dilucidar los hechos y otorgar herramientas mayores a los magistrados para resolver la causa.

Por otro lado, se mencionó que el Convenio Marco de Cooperación y Asistencia Técnica c

In [18]:
query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la tercera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
summary_3 = (index.query(query,llm=llm))
print(summary_3)

 En la sección "Sobre la tercera cuestión" del fallo del Tribunal de Mendoza, se analiza la admisión de la articulación presentada por el actor Dante Ramón Farconesi en su recurso extraordinario provincial contra la sentencia dictada en el auto n° 159.313. La cuestión en cuestión se refiere a la admisión de la defensa presentada por el Dr. Miguel Grosso, quien argumentó que la póliza entre el Gobierno de Mendoza y Provincia ART S.A. fue celebrada en fecha 01/09/2013.

El Tribunal encontró que la defensa presentada es relevante y admisible, ya que se trata de un hecho clave en el caso y puede afectar la resolución de la cuestión en disputa. Además, se consideró que la póliza es un documento probatorio que puede ser utilizado para respaldar la defensa presentada, por lo que se decidió admitirla en el proceso.

En resumen, la sección "Sobre la tercera cuestión" del fallo del Tribunal de Mendoza se centra en la admisión de la defensa presentada por el Dr. Miguel Grosso, quien argumentó que

In [20]:
query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'resuelve' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
summary_res = (index.query(query,llm=llm))
print(summary_res)

 En la sección "resuelve" del fallo del Tribunal de la Segunda Sala de la Suprema Corte de Justicia de Mendoza, se establece el dictamen final en la causa N° 13-04422907-7/1, conocida como "Farconesi, Dante Ramón c/ Provincia ART SA p/ Enfermedad Accidente". El Tribunal, after analyzing the arguments and evidence presented by both parties, rules in favor of the Province ART SA and dismisses the appeal filed by Dante Ramón Farconesi.

The Court finds that the lower court's decision was correct and well-founded, and that there is no grounds for annulling the sentence. The Court also imposes costs on the appellant for the unsuccessful appeal.

The ruling highlights the importance of respecting the decisions of the lower courts and the need to comply with the legal requirements for filing an appeal. The Court also emphasizes the need to carefully evaluate the evidence and arguments presented in an appeal, as the outcome of the case can depend on it.

In conclusion, the Tribunal's decision 

In [21]:
prompt = f"You are a member of the supreme court, Argentina. Given the following summary of a particular court ruling:\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
prompt +="Write a court rule summary in spanish in no less than 150 words. Do not mention:\n1. the name of the the sections.\n2. the numbers of the cases.\n3. the names of the ministers of the court.\n4. dates and court locations..\n\n"
prompt +="When writing the summary keep in mind it can help for future jurisprudence. Explain the causes behind the final decision"
print(prompt)

You are a member of the supreme court, Argentina. Given the following summary of a particular court ruling:

```
 En el "Antecedentes" de la sentencia en cuestión, se menciona que el informe pericial contable reveló que la póliza entre el Gobierno de Mendoza y Provincia ART S.A. fue celebrada el 1 de septiembre de 2013. Este dictamen fue considerado por ambas partes sin ser cuestionado por el actor. Además, se menciona que para otorgarles a los magistrados mayores herramientas para dilucidar los hechos, se celebró el "Convenio Marco de Cooperación y Asistencia Técnica" entre la Suprema Corte de Justicia, la Superintendencia de Riesgos del Trabajo y la Subsecretaría de Trabajo y Seguridad Social del Ministerio de Gobierno, Justicia y Derechos Humanos del Gobierno de Mendoza en fecha 17 de agosto de 2011. El aludido permite verificar la información existente en el organismo de contralor a través de su página institucional. A pesar de esto, la decisión de grado se sostiene sobre otros ele

In [22]:
print(llm(prompt,))

 El fallo del Tribunal de la Suprema Corte de Justicia de Mendoza en el caso "Farconesi, Dante Ramón c/ Provincia ART SA p/ Enfermedad Accidente" se centra en la admisión de la defensa presentada por el Dr. Miguel Grosso, quien argumentó que la póliza entre el Gobierno de Mendoza y Provincia ART S.A. fue celebrada en fecha 01/09/2013. El Tribunal encontró que la defensa presentada es relevante y admisible, ya que se trata de un hecho clave en el caso y puede afectar la resolución de la cuestión en disputa. Además, se consideró que la póliza es un documento probatorio que puede ser utilizado para respaldar la defensa presentada, por lo que se decidió admitirla en el proceso.

La sección "Sobre la segunda cuestión" del fallo analiza la admisión de la articulación interpuesta por Dante Ramón Farconesi en su recurso extraordinario provincial y se destaca la importancia de la cooperación y asistencia técnica entre los organismos para dilucidar los hechos y encontrar la verdad.

El Tribunal 

# FUNCTIONS

## Create summary

In [40]:
def create_summary(rule_file, embedding, llm):
    loader = UnstructuredPDFLoader(rule_file)
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    index.vectorstore.delete_collection()
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Antecedentes' of a particular court ruling . Write your answer in spanish in no less than 150 words:"
    summary_ant = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la primera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_1 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la segunda cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_2 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la tercera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_3 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'resuelve' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_res = (index.query(query,llm=llm))

    prompt = f"You are a member of the supreme court, Argentina. Given the following summary of a particular court ruling:\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
    prompt +="Write a court rule summary in spanish in no less than 150 words. Do not mention:\n1. the name of the the sections.\n2. the numbers of the cases.\n3. the names of the ministers of the court.\n4. dates and court locations..\n\n"
    prompt +="When writing the summary keep in mind it can help for future jurisprudence. Explain the causes behind the final decision. Translate your answer to spanish:"
    del index
    return llm(prompt)


In [49]:
test = create_summary("/root/JurisGPT/rawdata/laboral/10000003368.pdf", embedding = embedding, llm = llm)

In [50]:
test

' El Tribunal Superior de Mendoza en la causa "Manrique" (N° 13-04491180-3/1) analizó la interpretación de la normativa legal en relación al plazo de actuación para la resolución de causas laborales. Se destaca que el artículo 124 de la Ley nº 9.024 establece un plazo de 3 días hábiles judiciales para la apelación contra las Resoluciones Viales de los Jueces Administrativos, mientras que el artículo 21 de la Ley nº 8.729 establece un plazo de 10 días para la apelación de las decisiones del Organismo del Trabajo (Subsecretaría del Trabajo y Seguridad Social) en el procedimiento de conciliación laboral.\n\nLa Corte Suprema consideró que la ley 933 de la Provincia de Mendoza, que estableció un plazo de caducidad de 30 días para iniciar la acción de repetición por parte del acreedor, fue inconstitucional debido a que imponía un plazo inferior al establecido por el Código Civil. En este sentido, se resalta la importancia de la adhesión de la Provincia de Mendoza a las disposiciones contenid

## Create summary from dir

In [52]:
def create_summary_from_dir(directory_path, output_directory_path, embedding, llm):
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        # Extract the file extension
        name, extension = os.path.splitext(filename)
        
        # Build the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Skip directories, only work on PDF files
        if os.path.isfile(file_path) and extension.lower() == '.pdf':
            # Generate summary
            summary = create_summary(file_path, embedding, llm)
            
            # Save summary to new file
            summary_filename = f"{name}_summary.txt"
            summary_file_path = os.path.join(output_directory_path, summary_filename)
            
            with open(summary_file_path, 'w') as f:
                f.write(summary)



## Calculate ROUGE metric

In [72]:
import os
import re
from rouge import Rouge

def calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern):
    # Initialize Rouge
    rouge = Rouge()
    
    # Compile the regular expressions for filename patterns
    system_re = re.compile(system_filename_pattern)
    model_re = re.compile(model_filename_pattern)
    
    # Initialize storage for scores
    all_scores = []

    # Loop through the files in the system summaries directory
    for system_filename in os.listdir(system_dir):
        match = system_re.match(system_filename)
        if not match:
            continue

        # Extract ID
        system_id = match.groups()[0]

        # Find the corresponding model summary file using ID
        model_filename = f"{system_id}_summary.txt"
        model_path = os.path.join(model_dir, model_filename)

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found. Attempted to open {model_path}")
            continue

        

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found.")
            continue

        # Read system and model summaries
        with open(os.path.join(system_dir, system_filename), 'r', encoding='utf-8') as f:
            system_summary = f.read().strip()
        
        with open(model_path, 'r', encoding='utf-8') as f:
            model_summary = f.read().strip()

        # Compute the ROUGE score
        score = rouge.get_scores(system_summary, model_summary)[0]
        #all_scores.append(score)
        all_scores.append({
            'system_id': system_id,
            'rouge-1': score['rouge-1'],
            'rouge-2': score['rouge-2'],
            'rouge-l': score.get('rouge-l', {})  # Add this line to include ROUGE-L
        })
        # Compute the ROUGE score


        # Extract ROUGE-1, ROUGE-2, and ROUGE-3 scores
        #rouge_1 = score.get('rouge-1', {})
        #rouge_2 = score.get('rouge-2', {})
       
        # Pretty-print scores
        #print(f"ROUGE scores for ID {system_id}:")
        #print(f"  ROUGE-1: R: {rouge_1.get('r', 'N/A'):.4f}, P: {rouge_1.get('p', 'N/A'):.4f}, F: {rouge_1.get('f', 'N/A'):.4f}")
        #print(f"  ROUGE-2: R: {rouge_2.get('r', 'N/A'):.4f}, P: {rouge_2.get('p', 'N/A'):.4f}, F: {rouge_2.get('f', 'N/A'):.4f}")
      
        
        #print(f"ROUGE score for ID {system_id}: {score}")

    # If you want average scores, you can calculate that here from all_scores.
    # ...

    return all_scores


## Print ROUGE TABLE

In [81]:
import pandas as pd  # Don't forget to import pandas

def print_rouge_table(all_scores):
    pd.set_option('display.max_rows', None)
    
    system_ids = []
    metrics = []
    rs = []
    ps = []
    fs = []

    # Loop through each dictionary to extract the values
    for score_dict in all_scores:
        system_id = score_dict['system_id']

        for metric, values in score_dict.items():
            if metric == 'system_id':
                continue  # Skip the system_id, we've already saved it

            system_ids.append(system_id)
            metrics.append(metric)
            rs.append(values['r'])
            ps.append(values['p'])
            fs.append(values['f'])

    # Create the DataFrame
    df_long = pd.DataFrame({
        'system_id': system_ids,
        'metric': metrics,
        'r': rs,
        'p': ps,
        'f': fs
    })

    return df_long

# UTILS

## Stop LLAMA Jumpstart endpoint

In [75]:
import boto3

region = boto3.Session().region_name
endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-7b-f"

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '1bd5f1d8-6f85-414d-8591-3a4a6c1fc78a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1bd5f1d8-6f85-414d-8591-3a4a6c1fc78a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 19 Aug 2023 16:42:30 GMT'},
  'RetryAttempts': 0}}

# MAIN

## Create Summaries

In [ ]:
directory_path = "/root/JurisGPT/rawdata/laboral/"  
output_directory_path = "/root/JurisGPT/data/laboral/qa/"  
create_summary_from_dir(directory_path, output_directory_path, embedding = embedding, llm = llm)

## Calculate ROUGE

The ROUGE-1 metric is a type of Recall-Oriented Understudy for Gisting Evaluation (ROUGE) score that is commonly used for evaluating the quality of summaries by comparing them to reference summaries. Specifically, ROUGE-1 measures the overlap of unigrams (single words) between the generated summary and the reference summary.

The ROUGE-1 score comprises three values:

1. **Precision (P)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the generated summary. In essence, precision answers the question: "Of all the unigrams in the generated summary, how many are in the reference summary?"
   \[
   \text{Precision} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in generated summary}}
   \]

2. **Recall (R)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the reference summary. Recall answers the question: "Of all the unigrams in the reference summary, how many are in the generated summary?"
   \[
   \text{Recall} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in reference summary}}
   \]

3. **F1 Score (F)**: This is the harmonic mean of precision and recall and serves as a single number that balances the trade-off between precision and recall.
   \[
   \text{F1 Score} = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
   \]

ROUGE-1 is particularly useful for assessing the quality of extractive summaries, where sentences or phrases are selected from the original text to form a summary. The metric can also be used for abstractive summaries but may not capture the quality of the summary as effectively as when combined with other ROUGE metrics like ROUGE-2 (bigram overlap) or ROUGE-L (longest common subsequence).

In [82]:

# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'


# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [87]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003275,rouge-1,0.208333,0.102459,0.137363
3,10000003307,rouge-1,0.394737,0.092025,0.149254
6,10000003220,rouge-1,0.281553,0.224806,0.250000
9,10000003219,rouge-1,0.308943,0.111765,0.164147
12,10000003258,rouge-1,0.319149,0.173077,0.224439
15,10000003300,rouge-1,0.248062,0.132231,0.172507
18,10000003368,rouge-1,0.328000,0.172269,0.225895
21,10000003221,rouge-1,0.168000,0.108247,0.131661
24,10000003292,rouge-1,0.275168,0.127726,0.174468
27,10000003247,rouge-1,0.265487,0.147059,0.189274


In [85]:
rouge_table

,system_id,metric,r,p,f
0,10000003275,rouge-1,0.208333,0.102459,0.137363
1,10000003275,rouge-2,0.035354,0.017588,0.023490
2,10000003275,rouge-l,0.183333,0.090164,0.120879
3,10000003307,rouge-1,0.394737,0.092025,0.149254
4,10000003307,rouge-2,0.203540,0.041071,0.068351
5,10000003307,rouge-l,0.355263,0.082822,0.134328
6,10000003220,rouge-1,0.281553,0.224806,0.250000
7,10000003220,rouge-2,0.074830,0.059140,0.066066
8,10000003220,rouge-l,0.233010,0.186047,0.206897
9,10000003219,rouge-1,0.308943,0.111765,0.164147
